In [0]:
import os
import requests
from pyspark.sql import functions as F

In [0]:
# Diretório de destino
dir_path = "/Volumes/fomacao_microsoft_power_bi_profisional/bronze/landing/processar/"
os.makedirs(dir_path, exist_ok=True)

# URLs e nomes dos arquivos
files = {
    "CA Sales.csv": "https://raw.githubusercontent.com/JDLNas/curso_formacao_microsoft_powerbi_profissional/refs/heads/main/Fontes/Merge1/Pasta1/CA%20Sales.csv",
    "DE Sales.csv": "https://raw.githubusercontent.com/JDLNas/curso_formacao_microsoft_powerbi_profissional/refs/heads/main/Fontes/Merge1/Pasta1/DE%20Sales.csv",
    "FR Sales.csv": "https://raw.githubusercontent.com/JDLNas/curso_formacao_microsoft_powerbi_profissional/refs/heads/main/Fontes/Merge1/Pasta1/FR%20Sales.csv",
    "MX Sales.csv": "https://raw.githubusercontent.com/JDLNas/curso_formacao_microsoft_powerbi_profissional/refs/heads/main/Fontes/Merge1/Pasta1/MX%20Sales.csv"
}

# Baixar os arquivos
for file_name, url in files.items():
    file_path = os.path.join(dir_path, file_name)
    with open(file_path, "wb") as f:
        f.write(requests.get(url).content)

# Move cada arquivo baixado da pasta 'processar' para a pasta 'processados'
for file_name in files.keys():
    src_path = f"/Volumes/fomacao_microsoft_power_bi_profisional/bronze/landing/processar/{file_name}"  # Caminho de origem
    dst_path = f"/Volumes/fomacao_microsoft_power_bi_profisional/bronze/landing/processados/{file_name}"  # Caminho de destino
    dbutils.fs.mv(src_path, dst_path)  # Move o arquivo usando Databricks Utilities

In [0]:
# Caminhos dos arquivos CSV de vendas para cada país, utilizados como origem dos dados
origem_path_CA = "/Volumes/fomacao_microsoft_power_bi_profisional/bronze/landing/processados/CA Sales.csv"  # Canadá
origem_path_DE = "/Volumes/fomacao_microsoft_power_bi_profisional/bronze/landing/processados/DE Sales.csv"  # Alemanha
origem_path_FR = "/Volumes/fomacao_microsoft_power_bi_profisional/bronze/landing/processados/FR Sales.csv"  # França
origem_path_MX = "/Volumes/fomacao_microsoft_power_bi_profisional/bronze/landing/processados/MX Sales.csv"  # México

# Carrega o arquivo CSV de vendas do Canadá, aplicando opções de leitura e enriquecendo com metadados
df_raw_CA = (spark.read
          .option('header', 'true')      # Considera a primeira linha como cabeçalho
          .option('enconding', 'UTF-8')  # Define a codificação dos caracteres
          .option('sep', ',')            # Define o separador de colunas
          .option('inferSchema', 'true') # Infere automaticamente o tipo das colunas
          .option('multiline', 'true')   # Permite linhas com múltiplas quebras
          .option('mode', 'PERMISSIVE')  # Permite leitura mesmo com erros nos dados
          .csv(origem_path_CA)           # Caminho do arquivo fonte
          .withColumn('_ingest_ts_utc', F.current_timestamp())             # Timestamp de ingestão
          .withColumn('_ingest_date', F.to_date(F.col('_ingest_ts_utc')))  # Data de ingestão
          .withColumn('_source_path', F.col('_metadata.file_path'))        # Caminho do arquivo fonte
          .withColumn('_source_file', F.regexp_extract(F.col('_source_path'), r'([^/]+)$', 1))  # Nome do arquivo fonte
          .withColumnRenamed('Total Geral', 'Total_Geral'))  # Renomeia coluna para padrão sem espaço

# Carrega o arquivo CSV de vendas da Alemanha, aplicando opções de leitura e enriquecendo com metadados
df_raw_DE = (spark.read
          .option('header', 'true')
          .option('enconding', 'UTF-8')
          .option('sep', ',')
          .option('inferSchema', 'true')
          .option('multiline', 'true')
          .option('mode', 'PERMISSIVE')
          .csv(origem_path_DE)
          .withColumn('_ingest_ts_utc', F.current_timestamp())
          .withColumn('_ingest_date', F.to_date(F.col('_ingest_ts_utc')))
          .withColumn('_source_path', F.col('_metadata.file_path'))
          .withColumn('_source_file', F.regexp_extract(F.col('_source_path'), r'([^/]+)$', 1))
          .withColumnRenamed('Total Geral', 'Total_Geral'))

# Carrega o arquivo CSV de vendas da França, aplicando opções de leitura e enriquecendo com metadados
df_raw_FR = (spark.read
          .option('header', 'true')
          .option('enconding', 'UTF-8')
          .option('sep', ',')
          .option('inferSchema', 'true')
          .option('multiline', 'true')
          .option('mode', 'PERMISSIVE')
          .csv(origem_path_FR)
          .withColumn('_ingest_ts_utc', F.current_timestamp())
          .withColumn('_ingest_date', F.to_date(F.col('_ingest_ts_utc')))
          .withColumn('_source_path', F.col('_metadata.file_path'))
          .withColumn('_source_file', F.regexp_extract(F.col('_source_path'), r'([^/]+)$', 1))
          .withColumnRenamed('Total Geral', 'Total_Geral'))

# Carrega o arquivo CSV de vendas do México, aplicando opções de leitura e enriquecendo com metadados
df_raw_MX = (spark.read
          .option('header', 'true')
          .option('enconding', 'UTF-8')
          .option('sep', ',')
          .option('inferSchema', 'true')
          .option('multiline', 'true')
          .option('mode', 'PERMISSIVE')
          .csv(origem_path_MX)
          .withColumn('_ingest_ts_utc', F.current_timestamp())
          .withColumn('_ingest_date', F.to_date(F.col('_ingest_ts_utc')))
          .withColumn('_source_path', F.col('_metadata.file_path'))
          .withColumn('_source_file', F.regexp_extract(F.col('_source_path'), r'([^/]+)$', 1))
          .withColumnRenamed('Total Geral', 'Total_Geral'))

In [0]:
# Define os nomes das tabelas de destino para cada país
tbl_CA_Sales = "CA_Sales"
tbl_DE_Sales = "DE_Sales"
tbl_FR_Sales = "FR_Sales"
tbl_MX_Sales = "MX_Sales"

# Define os caminhos completos das tabelas Delta no esquema bronze
destino_path_CA_Sales = f'fomacao_microsoft_power_bi_profisional.bronze.{tbl_CA_Sales}'
destino_path_DE_Sales = f'fomacao_microsoft_power_bi_profisional.bronze.{tbl_DE_Sales}'
destino_path_FR_Sales = f'fomacao_microsoft_power_bi_profisional.bronze.{tbl_FR_Sales}'
destino_path_MX_Sales = f'fomacao_microsoft_power_bi_profisional.bronze.{tbl_MX_Sales}'

# Salva o DataFrame do Canadá como tabela Delta particionada por data de ingestão
df_bronze_CA_Sales = (df_raw_CA.write
                                  .format('delta')
                                  .mode('overwrite')
                                  .option('overwriteSchema', 'true')
                                  .partitionBy('_ingest_date')
                                  .saveAsTable(destino_path_CA_Sales))

# Salva o DataFrame da Alemanha como tabela Delta particionada por data de ingestão
df_bronze_DE_Sales = (df_raw_DE.write
                                  .format('delta')
                                  .mode('overwrite')
                                  .option('overwriteSchema', 'true')
                                  .partitionBy('_ingest_date')
                                  .saveAsTable(destino_path_DE_Sales))

# Salva o DataFrame da França como tabela Delta particionada por data de ingestão
df_bronze_FR_Sales = (df_raw_FR.write
                                  .format('delta')
                                  .mode('overwrite')
                                  .option('overwriteSchema', 'true')
                                  .partitionBy('_ingest_date')
                                  .saveAsTable(destino_path_FR_Sales))

# Salva o DataFrame do México como tabela Delta particionada por data de ingestão
df_bronze_MX_Sales = (df_raw_MX.write
                                  .format('delta')
                                  .mode('overwrite')
                                  .option('overwriteSchema', 'true')
                                  .partitionBy('_ingest_date')
                                  .saveAsTable(destino_path_MX_Sales))

# Exibe a contagem de registros para cada tabela Delta de vendas por país
print(f'Contagem de registros: {spark.table(destino_path_CA_Sales).count()}')  # Canadá
print(f'Contagem de registros: {spark.table(destino_path_DE_Sales).count()}')  # Alemanha
print(f'Contagem de registros: {spark.table(destino_path_FR_Sales).count()}')  # França
print(f'Contagem de registros: {spark.table(destino_path_MX_Sales).count()}')  # México